In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Fruityvice API

In [2]:
import requests
import json

### The fruityvice API data using "requests.get("url")" function. The data is in a json format.

In [3]:
data = requests.get("https://web.archive.org/web/20240929211114/https://fruityvice.com/api/fruit/all")

## Retrieve results using "json.loads()" function.

In [4]:
results = json.loads(data.text)

 ## Convert our json data into pandas data frame.

In [5]:
df = pd.DataFrame(results)
df.head()

,name,id,family,order,genus,nutritions
0,Persimmon,52,Ebenaceae,Rosales,Diospyros,"{'calories': 81, 'fat': 0.0, 'sugar': 18.0, 'c..."
1,Strawberry,3,Rosaceae,Rosales,Fragaria,"{'calories': 29, 'fat': 0.4, 'sugar': 5.4, 'ca..."
2,Banana,1,Musaceae,Zingiberales,Musa,"{'calories': 96, 'fat': 0.2, 'sugar': 17.2, 'c..."
3,Tomato,5,Solanaceae,Solanales,Solanum,"{'calories': 74, 'fat': 0.2, 'sugar': 2.6, 'ca..."
4,Pear,4,Rosaceae,Rosales,Pyrus,"{'calories': 57, 'fat': 0.1, 'sugar': 10.0, 'c..."


### The result is in a nested json format. The 'nutrition' column contains multiple subcolumns, so the data needs to be 'flattened' or normalized.

In [6]:
df2 = pd.json_normalize(results)
df2.head()

,name,id,family,order,genus,nutritions.calories,nutritions.fat,nutritions.sugar,nutritions.carbohydrates,nutritions.protein
0,Persimmon,52,Ebenaceae,Rosales,Diospyros,81,0.0,18.0,18.0,0.0
1,Strawberry,3,Rosaceae,Rosales,Fragaria,29,0.4,5.4,5.5,0.8
2,Banana,1,Musaceae,Zingiberales,Musa,96,0.2,17.2,22.0,1.0
3,Tomato,5,Solanaceae,Solanales,Solanum,74,0.2,2.6,3.9,0.9
4,Pear,4,Rosaceae,Rosales,Pyrus,57,0.1,10.0,15.0,0.4


### extract some information from this dataframe. Perhaps, we need to know the family and genus of a cherry.

#### * the family and genus of a cherry.

In [7]:
cherry = df2.loc[df2["name"] == 'Cherry']  # .loc means: “Give me the rows where this condition is True”
                                           # df2.loc[df2["name"] == "Cherry"] 

cherry.iloc[0]['family'], cherry.iloc[0]['genus'] # df.iloc[row_index] only single index. so ['family'] is attribut

('Rosaceae', 'Prunus')

### Option 1: One-liner (clean & professional)

In [8]:
family, genus = df2.loc[df2["name"] == "Cherry", ["family", "genus"]].values[0]
family, genus

('Rosaceae', 'Prunus')

### Option 2: Use semicolon (not recommended, but valid)

In [9]:
cherry = df2.loc[df2["name"] == "Cherry"]; (cherry.iloc[0]['family'], cherry.iloc[0]['genus'])

('Rosaceae', 'Prunus')

#### find out how many calories are contained in a banana.

In [10]:
cal_banana = df2.loc[df2["name"] == 'Banana']

cal_banana.iloc[0]['nutritions.calories']  # df.iloc[row_index]


np.int64(96)

In [11]:
banana_calories = df2.loc[df2["name"] == "Banana","nutritions.calories"]
banana_calories

2    96
Name: nutritions.calories, dtype: int64

In [12]:
banana_calories = df2.loc[df2["name"] == "Banana","nutritions.calories"].values[0] # Filtering usually returns a collection.values[0] extracts the actual value
banana_calories

# array([96])
# extract the first (and only) value. so values[0]

np.int64(96)